In [ ]:
from pathlib import Path
from bressen import read_directory, read_files
from bressen.vlakken import aggregate_peil

## Paden naar gegevens
Gegevens zijn aangeleverd in verschillende directories:
- 01_AGV_Dijktrajecten_Legger: dijktrajecten van waterschap Amstel Gooi en Vecht
- 02_ARK_Kering: kering van het Amsterdam-Rijnkanaal
- 03_Doorbraaklocaties: de locaties van de doorbraken
- 04_Watervlakken: alle beschikbare watervlakken

Hieronder staan alle paden naar de relevante directories en bestanden gespecificeerd waarmee we de klasse `BasisGegevens` kunnen samenstellen. Deze schrijven we weg in een GeoPackage die we later kunnen inlezen.

In [ ]:
DATA_DIR = Path(r"d:\projecten\D2405.waternet.bressen\01.gegevens")

PEILVAKKEN_DIR = DATA_DIR / "04_Peilvlakken"
WATERVLAKKEN_DIR = DATA_DIR / "05_Watervlakken"
KERINGEN_FILES = [
    DATA_DIR.joinpath("01_AGV_Dijktrajecten_Legger", "240122_Export_Output_11_verwerkt.shp"),
    DATA_DIR.joinpath("02_ARK_Kering", "240212_Waterkeringen_Blaeu_ARK.shp")
    ]

GPKG_OUT = DATA_DIR / "basisgegevens.gpkg"

## Peilvakken
Peilvakken samenstellen uit PEILVAKKEN_DIR. hierbij wordt een attribuut `peil` toegevoegd op basis van de minimale waarde uit een aantal kolommen met peilen

In [ ]:
# inlezen peilvakken
gdf = read_directory(PEILVAKKEN_DIR)

# peilkolommen
peil_kolommen = [
    'IWS_GPGVAS',
    'IWS_GPGOND',
    'IWS_GPGBOV',
    'VAST_PEIL',
    'FLEXIBEL_W',
    'FLEXIBEL_Z',
    'WINTERPEIL',
    'ZOMERPEIL'
    ]

# toevoegen kolom met peil
peilvlakken = aggregate_peil(gdf, columns=peil_kolommen, method="min", nodata=0)

## Watervlakken
Watervlakken lezen we in uit de directory met watervlakken

In [ ]:
watervlakken = read_directory(WATERVLAKKEN_DIR)

# Inlezen keringen
Keringen lezen uit verschillende bestanden.

Let op (!) voor het bouwen van bressen kunnen we geen MultiLineStrings gebruiken. Vandaar dat we deze omzetten naar (meerdere) LineStrings met `.explode()`

In [ ]:
keringen = read_files(KERINGEN_FILES)
keringen = keringen.explode(index_parts=False)
keringen.reset_index(inplace=True)